In [40]:
%%capture
# !pip install text_hammer

In [41]:
import os
import random
import torch
import wandb
import logging
import transformers
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from datasets import load_dataset
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn import metrics, model_selection, preprocessing
from transformers import AdamW, get_linear_schedule_with_warmup

import re
# import text_hammer as th
# from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer,TFBertModel

import warnings
warnings.filterwarnings("ignore",  category = FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [42]:
# wandb.login()

In [43]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # Some cudnn methods can be random even after fixing the seed 
    # unless you tell it to be deterministic
    torch.backends.cudnn.deterministic = True

seed_everything(1234)

In [44]:
# Loading datasets into DF
df = pd.read_csv("/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/emotion-data/goemotion_simplified.csv")

# Checking Shape
display(df.shape)

(43410, 33)

In [45]:
# df1 = pd.read_csv("/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/emotion-data/goemotions_1.csv", encoding='utf8')
# df2 = pd.read_csv("/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/emotion-data/goemotions_2.csv", encoding='utf8')
# df3 = pd.read_csv("/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/emotion-data/goemotions_3.csv", encoding='utf8')
# df4 = pd.read_csv("/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/DataSet/emotion-data/reddit_emotions_processed.csv", encoding='utf8')

In [46]:
# df = pd.concat([df1, df2, df3, df4], axis=0)

In [47]:
# from utils.preprocessing import expand_contractions
# df['text'] = df['text'].apply(expand_contractions)

In [48]:
df.head()

,Unnamed: 0.1,Unnamed: 0,text,labels,id,0,1,2,3,4,...,18,19,20,21,22,23,24,25,26,27
0,0,0,My favourite food is anything I didn't have to...,[27],eebbqej,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,"Now if he does off himself, everyone will thin...",[27],ed00q6i,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,WHY THE FUCK IS BAYLESS ISOING,[2],eezlygj,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,To make her feel threatened,[14],ed7ypvh,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,Dirty Southern Wankers,[3],ed0bdzj,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
from utils.preprocessing import get_clean_dataset
df = get_clean_dataset(df)

In [50]:
df.head()

,Unnamed: 0.1,Unnamed: 0,text,labels,id,0,1,2,3,4,...,18,19,20,21,22,23,24,25,26,27
0,0,0,favourite food anything didnt have cook myself,[27],eebbqej,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,1,does himself everyone will think having laugh ...,[27],ed00q6i,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2,2,FUCK BAYLESS ISOING,[2],eezlygj,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,3,make feel threatened,[14],ed7ypvh,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,4,Dirty Southern Wankers,[3],ed0bdzj,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
from sklearn.model_selection import train_test_split
train_temp, test = train_test_split(df, test_size=0.2, shuffle=True, random_state=42)

In [52]:
train, valid = train_test_split(train_temp, test_size=0.2, shuffle=True, random_state=42)

In [53]:
# train[[str(i) for i in range(0, 28)]].values.tolist()

🧹Clean Data (Noise Entity Removal) ¶

In [54]:
# def text_preprocessing(df,col_name):
#     tqdm.pandas()
#     df[col_name] = df[col_name].progress_apply(lambda x:str(x).lower())
#     df[col_name] = df[col_name].progress_apply(lambda x: th.remove_emails(x))
#     df[col_name] = df[col_name].progress_apply(lambda x: th.remove_html_tags(x))
#     df[col_name] = df[col_name].progress_apply(lambda x: th.remove_urls(x))
#     df[col_name] = df[col_name].progress_apply(lambda x: th.remove_special_chars(x))
#     df[col_name] = df[col_name].progress_apply(lambda x: th.remove_accented_chars(x))
#     df[col_name] = df[col_name].progress_apply(lambda text: th.cont_exp(text))
#     df[col_name] = df[col_name].progress_apply(lambda x: re.sub("[" 
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         "]+", "", x))
#     df[col_name] = df[col_name].progress_apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))
#     df[col_name] = df[col_name].progress_apply(lambda x: ' '.join(x.split()))
    
#     return(df)

In [55]:
# # %%capture
# train = text_preprocessing(train,'text')
# valid = text_preprocessing(valid,'text')
# test = text_preprocessing(test,'text')

🗑️Remove Stop Words ¶

In [56]:
# stop = stopwords.words('english')
# train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# valid['text'] = valid['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
# test['text'] = test['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

🌱Lemmatization ¶

In [57]:
# nltk.download('wordnet')
# !unzip "/usr/share/nltk_data/corpora/wordnet.zip" -d "/usr/share/nltk_data/corpora/"

In [58]:
# def word_lemmatizer(text):
#     lemmatizer = WordNetLemmatizer()
#     return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

# train['text'] = train['text'].apply(lambda text: word_lemmatizer(text))
# valid['text'] = valid['text'].apply(lambda text: word_lemmatizer(text))

In [59]:
def get_class_numbers():
    return [str(i) for i in range(28)]

In [60]:
from transformers import AutoModelForSequenceClassification
from utils.dataset import Dataset

def build_dataset(tokenizer_max_len, tokenizer):
    train_dataset = Dataset(train.text.tolist(), train[get_class_numbers()].values.tolist(), tokenizer, tokenizer_max_len)
    valid_dataset = Dataset(valid.text.tolist(), valid[get_class_numbers()].values.tolist(), tokenizer, tokenizer_max_len)
    test_dataset = Dataset(test.text.tolist(), test[get_class_numbers()].values.tolist(), tokenizer, tokenizer_max_len)

    return train_dataset, valid_dataset, test_dataset
    # return train_dataset, valid_dataset

def build_dataloader(train_dataset, valid_dataset, test_dataset, batch_size):
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    valid_data_loader = DataLoader(valid_dataset, batch_size=int(batch_size/2), shuffle=True, num_workers=1)
    test_data_loader = DataLoader(test_dataset, batch_size=len(test_dataset), shuffle=True, num_workers=1)

    return train_data_loader, valid_data_loader, test_data_loader
    # return train_data_loader, valid_data_loader

def ret_model(do_prob, model_ckpt, n_labels, is_freez):
    # model = TextClassification(n_classes=n_labels, dropout=do_prob, model_ckpt=model_ckpt)
    model = AutoModelForSequenceClassification.from_pretrained(model_ckpt, num_labels=n_labels)
    if model_ckpt != 'roberta-large' and is_freez==True:
        # Freeze all layers except the classification layer
        for param in model.base_model.parameters():
            param.requires_grad = False

        # Modify the classification layer for the new task (target domain)
        model.classifier = torch.nn.Linear(model.classifier.in_features, n_labels)
        
    return model

In [61]:
def ret_optimizer(model):
    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    # opt = AdamW(optimizer_parameters, lr=wandb.config.learning_rate)
    opt = AdamW(model.parameters(), lr=wandb.config.learning_rate)
    return opt

In [62]:
def ret_scheduler(optimizer, num_train_steps):
    scheduler  = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=num_train_steps)
    return scheduler 

In [63]:
import torch
import torch.nn.functional as F

def loss_fn(y_pred, y_true, loss_func, alpha=0.25, gamma=2.0):
    
    if y_true is None:
        return None
    
    if loss_func == 'cross_entropy':
        return F.cross_entropy(y_pred, y_true.float())
    if loss_func == 'focal_loss':
        logpt = -F.binary_cross_entropy_with_logits(y_pred, y_true.float())
        pt = torch.exp(logpt)
        loss = -((1 - pt) ** gamma) * logpt
        return loss.mean()
    if loss_func == 'hinge_loss':
        return F.hinge_embedding_loss(y_pred, y_true.float())
    if loss_func == 'mse_loss':
         return nn.MSELoss()(y_pred, y_true.float())
    
    if loss_func == 'multi_label_soft_margin':
        return nn.MultiLabelSoftMarginLoss()(y_pred, y_true.float())
    
    # if not all others
    return nn.BCEWithLogitsLoss()(y_pred, y_true.float())

In [64]:
def train_fn(data_loader, model, optimizer, device, scheduler):
    train_loss = 0.0
    model.train()
    for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
        ids = d["ids"]
        mask = d["mask"]
        targets = d["labels"]
        # token_type_ids = d['token_type_ids']

        ids = ids.to(device, dtype=torch.long)
        mask = mask.to(device, dtype=torch.long)
        targets = targets.to(device, dtype=torch.float)
        
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        outputs = model(ids, attention_mask=mask, labels=targets)
        # loss = outputs.loss

        loss = loss_fn(y_pred=outputs.logits, y_true=targets, 
                           loss_func=get_loss_func())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        train_loss += loss.item()
        
    return train_loss
    

def eval_fn(data_loader, model, device):
    eval_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    val_accuracy = 0
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]
            # token_type_ids = d['token_type_ids']

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids, attention_mask=mask, labels=targets)
            loss = loss_fn(y_pred=outputs.logits, y_true=targets, 
                           loss_func=get_loss_func())

            eval_loss += loss.item()
            
            fin_targets.extend(targets)
            fin_outputs.extend(torch.nn.functional.softmax(outputs.logits))
            
    return eval_loss, fin_outputs, fin_targets

def test_fn(data_loader, model, device):
    test_loss = 0.0
    model.eval()
    fin_targets = []
    fin_outputs = []
    test_acc = 0
    with torch.no_grad():
        for bi, d in tqdm(enumerate(data_loader), total=len(data_loader)):
            ids = d["ids"]
            mask = d["mask"]
            targets = d["labels"]
            # token_type_ids = d['token_type_ids']

            ids = ids.to(device, dtype=torch.long)
            mask = mask.to(device, dtype=torch.long)
            targets = targets.to(device, dtype=torch.float)

            outputs = model(ids, attention_mask=mask, labels=targets)
            loss = loss_fn(y_pred=outputs.logits, y_true=targets, 
                           loss_func=get_loss_func())
#             logits = outputs.logits
            
            test_loss += loss.item()
            
            fin_targets.extend(targets)
            fin_outputs.extend(torch.nn.functional.softmax(outputs.logits))

    return test_loss, fin_outputs, fin_targets

In [65]:
# n_labels = 8
def model_info(model_ckpt):
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_ckpt, do_lower_case=True)
    bert_model = transformers.AutoModel.from_pretrained(model_ckpt)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    return bert_model, tokenizer, device

In [66]:
# using time module
import time
from datetime import datetime  

current_time = datetime.now()
time_stamp = current_time.timestamp()

date_time = datetime.fromtimestamp(time_stamp)
str_date_time = date_time.strftime("%d_%m_%Y-%H_%M_%S")

# dir = './bert_model_save'
output_dir = './model_save/emotion_pred/BERT/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [67]:
from utils.common_functions import log_metrics

training_stats = []
test_stats = []
def trainer(config=None):
    with wandb.init(config=config):
        config = wandb.config

        _, tokenizer, device = model_info(get_model_ckpt())

        train_dataset, valid_dataset, test_dataset = build_dataset(config.tokenizer_max_len, tokenizer)
        train_data_loader, valid_data_loader, test_data_loader = build_dataloader(train_dataset, valid_dataset, test_dataset, config.batch_size)
        
        print("Length of Train Dataloader: ", len(train_data_loader))
        print("Length of Valid Dataloader: ", len(valid_data_loader))
#         print("Length of Test Dataloader: ", len(test_data_loader))

        n_train_steps = int(len(train_dataset) / config.batch_size * 10)

        model = ret_model(config.dropout, get_model_ckpt(), get_n_labels(), is_freez=get_Freeze())
        optimizer = ret_optimizer(model)
        scheduler = ret_scheduler(optimizer, n_train_steps)
        model.to(device)
        # model = nn.parallel.DistributedDataParallel(model)
        wandb.watch(model)
        
        n_epochs = config.epochs

        best_val_loss = 100
        for epoch in tqdm(range(n_epochs)):
            train_loss = train_fn(train_data_loader, model, optimizer, device, scheduler)
            eval_loss, preds, labels = eval_fn(valid_data_loader, model, device)
          
            metrics = log_metrics(preds, labels)
            
            avg_train_loss, avg_val_loss = train_loss / len(train_data_loader), eval_loss / len(valid_data_loader)

            wandb.log({
                "epoch": epoch + 1,
                "train_loss": avg_train_loss,
                "val_loss": avg_val_loss,
                "auc_score": metrics['auc'],
            })

            print("AUC score: ", metrics['auc'])
            print("Average Train loss: ", avg_train_loss)
            print("Average Valid loss: ", avg_val_loss)
            print("Valid F1: ", metrics['f1'])
            print("Valid Acc: ", metrics['acc'])

            training_stats.append(
            {
                'Batch': config.batch_size,
                'Max Token': config.tokenizer_max_len,
                'Model':  get_model_ckpt(),
                'LR': config.learning_rate,
                'Loss func': get_loss_func(),
                'Epoch': epoch + 1,
                'Train Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. AUC': metrics['auc'],
                'F1 score': metrics['f1'],
                'Recall': metrics['recall'],
                'Precision': metrics['precision'],
                'Accuracy': metrics['acc'],
            })

            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                torch.save(model, output_dir+f"best_{get_type()}_{get_model_ckpt()}-{str(str_date_time)}.pt")
                tokenizer.save_pretrained(output_dir)
                print("Model saved as current val_loss is: ", best_val_loss)
            
        test_loss, preds, labels = test_fn(test_data_loader, model, device)
        metrics = log_metrics(preds, labels)
        avg_test_loss = test_loss / len(test_data_loader)
        print("Test loss: ", avg_test_loss)
        print("AUC: ", metrics['auc'])
        print("Test F1: ", metrics['f1'])
        print("Test Recall: ", metrics['recall'])
        print("Test Precision: ", metrics['precision'])
        print("Test Acc: ", metrics['acc'])
        test_stats.append({
                'Batch': config.batch_size,
                'Max Token': config.tokenizer_max_len,
                'Model':  get_model_ckpt(),
                'LR': config.learning_rate,
                'Loss func': get_loss_func(),
                'Test. Loss': avg_test_loss,
                'Test. AUC': metrics['auc'],
                'F1 score': metrics['f1'],
                'Recall': metrics['recall'],
                'Precision': metrics['precision'],
                'Accuracy': metrics['acc']
                })

In [68]:
def get_project():
    return 'emotion-prediction'

In [69]:
sweep_config = {
    'method': 'grid', #grid, random, bayesian
    'metric': {
      'name': 'auc_score',
      'goal': 'maximize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [2e-05, 3e-05, 5e-05]
        },
        'batch_size': {
            'values': [16]
        },
        'epochs':{'value': 3},
        'dropout':{
            'values': [0.3]
        },
        'tokenizer_max_len': {
            'values': [30, 70]
        },
    }
}

sweep_id = wandb.sweep(sweep_config, project=get_project())

Create sweep with ID: annlix9l
Sweep URL: https://wandb.ai/supreme-lab/emotion-prediction/sweeps/annlix9l


In [70]:
def get_Freeze():
    return False

def get_model_ckpt():
    #  return "bsingh/roberta_goEmotion"
    #  return 'distilbert-base-uncased'
    return 'bert-base-uncased'
    # return 'bert-large-uncased'
    #  return 'cardiffnlp/twitter-roberta-base-emotion-multilabel-latest'
    #  return 'roberta-large'

def get_loss_func():
    # return 'hinge_loss'
    # return 'focal_loss'
    # return 'cross_entropy'
    # return 'mse_loss'
    # return 'multi_label_soft_margin'
    return 'bce_loss'

def get_classes():
    trends = ["approval","toxic","obscene", 'insult', "threat", "hate", "offensive", "neither"]
    emotions = [
        'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring',
        'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval',
        'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief',
        'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization',
        'relief', 'remorse', 'sadness', 'surprise', 'neutral'
    ]
    # return range(0, 28)
    return emotions
def get_n_labels():
    return len(get_classes())

def get_type():
    return 'emotion'
#     return 'emotion'

In [71]:
# run the sweep
wandb.agent(sweep_id, function=trainer, count=6)

wandb: Agent Starting Run: wzh3x5nr with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.3
wandb: 	epochs: 3
wandb: 	learning_rate: 2e-05
wandb: 	tokenizer_max_len: 30
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Length of Train Dataloader:  1737
Length of Valid Dataloader:  869


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

...

  0%|          | 0/869 [00:00<?, ?it/s]

AUC score:  0.9141085717807652
Average Train loss:  0.0854949074087983
Average Valid loss:  0.1043768652786594
Valid F1:  0.7508656499881212
Valid Acc:  0.9626249434412406
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  0%|          | 0/1 [00:00<?, ?it/s]

Test loss:  0.10489863157272339
AUC:  0.913872085363516
Test F1:  0.7488195282351071
Test Recall:  0.7317686710544002
Test Precision:  0.7690049129059349
Test Acc:  0.9622782768947247


auc_score,▁▇█
epoch,▁▅█
train_loss,█▃▁
val_loss,█▁▃
auc_score,0.91411
epoch,3
train_loss,0.08549
val_loss,0.10438


## Print the Training loss, Validation loss and AUC values

In [ ]:
import pandas as pd

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Display the table.
df_stats

""


In [ ]:
df_stats.to_csv(f'/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/Main-Implementation/logs/emotion_result/train_valid_log_emotion_{get_model_ckpt()}-non-freeze.csv')

In [ ]:
import pandas as pd

# Display floats with two decimal places.
# pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=test_stats)

# Use the 'epoch' as the row index.
# df_stats = df_stats.set_index('Epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

""


In [ ]:
df_stats.to_csv(f'/home/siu856533724/code/source-code/Social-Networks/Trend-Prediction/Main-Implementation/logs/emotion_result/test_log_emotion_{get_model_ckpt()}-non-freeze.csv')

## Test BERT model using unknown data

In [ ]:
from utils.bert_classifier import get_model_tokenizer, Tokenize, Classification
# Test sentiment prediction
test_text = "I love you"
model, tokenizer = get_model_tokenizer(pred_type='emotion_pred', mode_pt_file='best_emotion_bert-large-uncased-11_12_2023-13_56_57.pt', 
                                       model_ckpt='bert-large-uncased', labels=28, is_freez=1, gpu='cuda')
dict = Tokenize(text=test_text, max_length=30, tokenizer=tokenizer)
scores = Classification(dict, model, gpu='cuda')
print(f"Predicted Outputs: {scores}")

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint a

Predicted Outputs: [[0.04972662404179573, 0.05081344395875931, 0.031225554645061493, 0.03223400190472603, 0.043478235602378845, 0.03591315075755119, 0.03896919637918472, 0.03507215157151222, 0.03231365978717804, 0.044278476387262344, 0.02880162186920643, 0.029653994366526604, 0.02706073597073555, 0.0249404925853014, 0.02791152521967888, 0.03434571996331215, 0.022936025634407997, 0.04122310131788254, 0.04890765622258186, 0.028191369026899338, 0.037287529557943344, 0.023911183699965477, 0.03755311295390129, 0.029031742364168167, 0.02581167034804821, 0.030616171658039093, 0.03558940440416336, 0.0722024217247963]]
